# PReLU (CNN & Object Detection) — Uçtan Uca Notlar

Bu defter, **PReLU’nun ne olduğunu** en temelden başlayıp pratikte **CNN ve object detection** tarafında doğru yerde nasıl kullanılacağını anlatır.  
Kod örnekleri **minimal** tutulmuştur. Matematiğe **bilerek girmiyoruz** (ayrı defterde).

> Hedef: “Ne zaman PReLU?”, “LeakyReLU’dan farkı ne?”, “Nerede iş yapar / nerede gereksiz?”

## 1) ReLU ve LeakyReLU’yu 1 cümlede hatırlayalım

- **ReLU:** negatif tarafta tamamen 0’a keser  
- **LeakyReLU:** negatif tarafta küçük bir sızıntı (sabit eğim) bırakır: `negative_slope = α`

Bu iki yaklaşımın ortak problemi:  
**Negatif taraftaki davranış sabittir** (ya 0 ya da seçtiğin α).

## 2) PReLU fikri: “Negatif eğimi model öğrensin”

PReLU = *Parametric ReLU*.

LeakyReLU’da negatif eğim **senin verdiğin sabit** bir sayıdır.  
PReLU’da negatif eğim **öğrenilebilir bir parametredir**.

Bu şu anlama gelir:
- Model, veriye göre “negatif tarafta ne kadar sızdıracağını” kendi ayarlar.
- Bazı görevlerde LeakyReLU’dan daha esnek ve daha güçlü olabilir.

Ama bedeli var:
- Ek parametre (küçük ama var)
- Overfitting riski (özellikle küçük dataset / çok parametre)

In [1]:
import torch
import torch.nn as nn

prelu = nn.PReLU(num_parameters=1, init=0.25)  # tek parametre (shared)
x = torch.tensor([-3.0, -1.0, 0.0, 1.0, 3.0])
print("x    :", x.tolist())
print("PReLU:", prelu(x).tolist())
print("learnable a:", prelu.weight.detach().cpu().tolist())

x    : [-3.0, -1.0, 0.0, 1.0, 3.0]
PReLU: [-0.75, -0.25, 0.0, 1.0, 3.0]
learnable a: [0.25]


## 3) PReLU parametresi: “a” nasıl çalışır?

PyTorch’ta PReLU’nun öğrenilebilir parametresi `weight` olarak tutulur.  
Bu parametre genelde “a” olarak anılır.

- **a küçükse** → ReLU’ya yaklaşır (negatifleri daha çok bastırır)
- **a büyükse** → negatiften daha çok bilgi geçirir (daha lineer)

Önemli: PReLU’da `a` eğitim boyunca optimizer ile güncellenir.

## 4) “Shared” mi “Channel-wise” mı?

PReLU’nun iki yaygın kullanımı var:

### A) Shared PReLU (tek parametre)
- Tüm kanallar aynı `a` değerini paylaşır
- Parametre sayısı çok az
- Daha az overfit

PyTorch:
```python
nn.PReLU(num_parameters=1)
```

### B) Channel-wise PReLU (kanal başına parametre)
- Her kanalın ayrı `a` parametresi olur
- Daha esnek (daha yüksek kapasite)
- Parametre sayısı artar

PyTorch:
```python
nn.PReLU(num_parameters=C)
```

CNN/backbone içinde çoğu zaman kanal-başı PReLU daha “mantıklı” görünür, ama veri küçükse shared tercih edilebilir.

In [2]:
import torch
import torch.nn as nn

C = 16
x = torch.randn(2, C, 8, 8)

prelu_shared = nn.PReLU(num_parameters=1, init=0.25)
prelu_channel = nn.PReLU(num_parameters=C, init=0.25)

y1 = prelu_shared(x)
y2 = prelu_channel(x)

print("shared weight shape :", prelu_shared.weight.shape)
print("channel weight shape:", prelu_channel.weight.shape)
print("output shapes:", y1.shape, y2.shape)

shared weight shape : torch.Size([1])
channel weight shape: torch.Size([16])
output shapes: torch.Size([2, 16, 8, 8]) torch.Size([2, 16, 8, 8])


## 5) CNN ve Object Detection’da nerede kullanılır?

### Backbone (özellik çıkarıcı)
PReLU backbone’da güçlü bir opsiyondur:
- Derin ağlarda gradient akışını iyileştirebilir
- Bazı mimarilerde doğruluğu artırabilir

### Neck (FPN / PAN vb.)
Kullanılır ama genelde “default” değildir.  
Modern YOLO tarafında SiLU daha standarttır.  
PReLU, daha çok “backbone kapasitesini artırma” denemelerinde öne çıkar.

### Head
Head’de kullanmak mümkün ama çoğu sistem:
- hız/kararlılık için SiLU/LeakyReLU
- veya baseline ReLU
seçer.

**Pratik kural:**
- “Kapasite + esneklik” istiyorsan → PReLU (özellikle backbone)
- “Hız + basitlik” istiyorsan → ReLU / LeakyReLU / SiLU

## 6) PReLU ne zaman mantıklı?

Aşağıdaki durumlarda PReLU denemek mantıklıdır:

- Model derin ve karmaşıksa
- Veride dağılım kaymaları varsa (domain shift)
- ReLU/LeakyReLU ile yakınsama sorunları görüyorsan
- Backbone tarafında “az ama anlamlı” iyileştirme arıyorsan

Ve özellikle:
- Ablation yapmayı planlıyorsan (aktivasyon değişimi tek başına)

## 7) PReLU’nun riskleri (net)

PReLU “bedava” değil:

- Kanal başına PReLU, parametre sayısını artırır (küçük görünür ama vardır)
- Küçük dataset’te overfit riskini artırabilir
- Bazı deployment/quantization akışlarında ekstra dikkat ister
- Bazen kazanım yoktur (ya da çok küçük olur)

Bu yüzden PReLU genelde:
- **Research/ablation**
- **backbone güçlendirme**
senaryolarında daha iyi bir seçimdir.

## 8) PyTorch’ta pratik kullanım (Conv + Norm + PReLU)

Aşağıdaki blok CNN’de klasik formdur:
**Conv → Norm → PReLU**

Not: PReLU parametresi normal activation gibi “stateless” değildir; öğrenilir.

In [3]:
import torch
import torch.nn as nn

class ConvBNPReLU(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1, per_channel=True, init=0.25):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        num_p = cout if per_channel else 1
        self.act = nn.PReLU(num_parameters=num_p, init=init)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

x = torch.randn(2, 16, 32, 32)
block = ConvBNPReLU(16, 32, per_channel=True)
y = block(x)
print(y.shape, "PReLU params:", block.act.weight.numel())

torch.Size([2, 32, 32, 32]) PReLU params: 32


## 9) “PReLU gerçekten öğreniyor mu?” mini kontrol

Aşağıdaki mini örnek, birkaç adımda `a` parametresinin değişebildiğini gösterir.
Bu bir demo; gerçek eğitim gibi düşünme.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0)

prelu = nn.PReLU(num_parameters=1, init=0.25)
lin = nn.Linear(4, 4)

model = nn.Sequential(lin, prelu)
opt = optim.SGD(model.parameters(), lr=0.1)

x = torch.randn(32, 4)
target = torch.zeros_like(x)  # sadece demo

print("a before:", prelu.weight.item())

for step in range(5):
    opt.zero_grad(set_to_none=True)
    out = model(x)
    loss = (out - target).pow(2).mean()
    loss.backward()
    opt.step()

print("a after :", prelu.weight.item())

a before: 0.25
a after : 0.20293855667114258


## 10) LeakyReLU ile karşılaştırma (tek paragraf)

- **LeakyReLU:** negatif eğim sabit → basit, stabil, hızlı  
- **PReLU:** negatif eğim öğrenilir → daha esnek, bazen daha iyi performans, ama overfit/karmaşıklık riski

Bu yüzden repo tasarımında mantıklı ayrım:
- LeakyReLU → **Core**
- PReLU → **Core ama “learnable activation” notu** (veya “Core/Advanced” gibi)

## 11) Mini kontrol listesi ✅

PReLU eklerken:
- [ ] shared mi channel-wise mı karar ver
- [ ] init değerini belirle (genelde 0.25 iyi başlangıç)
- [ ] aynı mimaride sadece aktivasyonu değiştirip ablation yap
- [ ] küçük dataset’te overfit izlerini kontrol et
- [ ] deployment/quantization planın varsa erkenden test et

Bu kadar. Matematik ve türev/backprop kısmını ayrı defterde ele alacağız.